## **Data Ingestion**

In [21]:
import os
import pandas as pd
import numpy as np

from pptx import Presentation
from pptx.chart.data import CategoryChartData
from datetime import datetime

pd.options.display.max_columns = 30
pd.options.display.float_format = '{:.2f}'.format


try:
    base_dir = os.path.dirname(os.path.abspath(__file__))  # quando for script .py
except NameError:
    base_dir = os.getcwd()  # quando for Jupyter/console interativo

# file_path = os.path.join(base_dir, "dados", "vendas_produtos.xlsx")
file_path = r"data\vendas_produtos.xlsx" 
df = pd.read_excel(file_path)

In [66]:
# Filtros fixos
FILTERS = {
"formats": "T. Embalagens",
"region": "Total Brasil",
"channels": "Hipermercados",
"periods": ['YTD Dec-20','YTD Dec-21','YTD Dec-22','YTD Dec-23'],
"brands": ['Coca-Cola Cia', 'Ambev Cia', 'B-Brands (Outros)'] 
}

# Colunas que queremos utilizar para gerar o filtro filnal
QUERY_COLUMNS = ['Brands','Channels', 'Periods']

VARIABLES_NAME = {
"variable_01": 'Value_(R$)',
}

variable_01 = VARIABLES_NAME.get('variable_01')

ALL_COLUMNS = [*QUERY_COLUMNS,variable_01]


In [69]:
df_share = df[
    (df['Formats']==FILTERS.get('formats')) &
    (df['Region']==FILTERS.get('region')) &
    (df['Channels']==FILTERS.get('channels')) &
    (df['Periods'].isin(FILTERS.get('periods'))) &
    (df['Brands'].isin(FILTERS.get('brands')))
]

In [71]:
df_total = df[
    (df['Formats']==FILTERS.get('formats')) &
    (df['Region']==FILTERS.get('region')) &
    (df['Channels']==FILTERS.get('channels')) &
    (df['Periods'].isin(FILTERS.get('periods'))) &
    (df['Brands'].isin(['T. Fabricantes'])) # Linha que irá gerar o totalizador
]

In [76]:
# Agrupa por marca e canal, somando os valores
df_grouped = df_share.groupby(QUERY_COLUMNS)[variable_01].sum().reset_index()
df_grouped_total = df_total.groupby(QUERY_COLUMNS)[variable_01].sum().reset_index()

In [77]:
df_grouped.sort_values(by=QUERY_COLUMNS, ascending=[True, True, True], inplace=True)

In [93]:
df_merged = df_grouped.merge(
    df_grouped_total,
    on=['Channels','Periods'],
    suffixes=('', '_Total')
)

In [94]:
# Calcular o share
df_merged['Share_(%)'] = (df_merged['Value_(R$)'] / df_merged['Value_(R$)_Total'] * 100).round(2)

In [103]:
df_merged.sort_values(by=QUERY_COLUMNS, ascending=[False, False, True], inplace=True)

In [ ]:
df_merged['Ano'] = df_merged['Periods'].str.extract(r'(\d{2})$').astype(int)

,Brands,Channels,Periods,Value_(R$),Share_(%),Brands_Total,Value_(R$)_Total,Ano
8,Coca-Cola Cia,Hipermercados,YTD Dec-20,337875400.00,64.55,T. Fabricantes,523423600.00,20
9,Coca-Cola Cia,Hipermercados,YTD Dec-21,336037100.00,64.21,T. Fabricantes,523337500.00,21
10,Coca-Cola Cia,Hipermercados,YTD Dec-22,304274000.00,64.68,T. Fabricantes,470428200.00,22
11,Coca-Cola Cia,Hipermercados,YTD Dec-23,319333100.00,62.80,T. Fabricantes,508515400.00,23
4,B-Brands (Outros),Hipermercados,YTD Dec-20,69870500.00,13.35,T. Fabricantes,523423600.00,20
5,B-Brands (Outros),Hipermercados,YTD Dec-21,58739250.00,11.22,T. Fabricantes,523337500.00,21
6,B-Brands (Outros),Hipermercados,YTD Dec-22,73424080.00,15.61,T. Fabricantes,470428200.00,22
7,B-Brands (Outros),Hipermercados,YTD Dec-23,83255350.00,16.37,T. Fabricantes,508515400.00,23
0,Ambev Cia,Hipermercados,YTD Dec-20,115677700.00,22.10,T. Fabricantes,523423600.00,20
1,Ambev Cia,Hipermercados,YTD Dec-21,128561200.00,24.57,T. Fabricantes,523337500.00,21


In [108]:
df_merged.sort_values(by='Ano', ascending=True, inplace=True)
df_merged


,Brands,Channels,Periods,Value_(R$),Share_(%),Brands_Total,Value_(R$)_Total,Ano
8,Coca-Cola Cia,Hipermercados,YTD Dec-20,337875400.00,64.55,T. Fabricantes,523423600.00,20
4,B-Brands (Outros),Hipermercados,YTD Dec-20,69870500.00,13.35,T. Fabricantes,523423600.00,20
0,Ambev Cia,Hipermercados,YTD Dec-20,115677700.00,22.10,T. Fabricantes,523423600.00,20
5,B-Brands (Outros),Hipermercados,YTD Dec-21,58739250.00,11.22,T. Fabricantes,523337500.00,21
1,Ambev Cia,Hipermercados,YTD Dec-21,128561200.00,24.57,T. Fabricantes,523337500.00,21
9,Coca-Cola Cia,Hipermercados,YTD Dec-21,336037100.00,64.21,T. Fabricantes,523337500.00,21
6,B-Brands (Outros),Hipermercados,YTD Dec-22,73424080.00,15.61,T. Fabricantes,470428200.00,22
10,Coca-Cola Cia,Hipermercados,YTD Dec-22,304274000.00,64.68,T. Fabricantes,470428200.00,22
2,Ambev Cia,Hipermercados,YTD Dec-22,92730070.00,19.71,T. Fabricantes,470428200.00,22
11,Coca-Cola Cia,Hipermercados,YTD Dec-23,319333100.00,62.80,T. Fabricantes,508515400.00,23
